<a href="https://colab.research.google.com/github/adamggibbs/marine-carbonate-system-ml-prediction/blob/master/data_cleaning_glider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning and Preprocessing
This notebook contains code to parse data from shipboard and glider data files to create desired datasets for machine learning training purposes.

## Data Storage
The data is stored in google drive. The first code cell mounts your google drive to the notebook for later use. The data directory is then specified. The Data directory must contains two folder labeled ship and glider. Within the ship folder are .csv files with shipboard data and within the glider folder are tab separated .txt files with glider data.

## Parsing Functions
The notebook contains two functions. The first parses the shipboard data and the second parses the glider data. Both create a numpy array with only quality checked data points desired for input to the algorithms being tested. Note that when using the glider function, the standard glider text file can be used without any further manipulation. However, all lines above the header of the shipboard data files should be removed and the file should be saved as a .csv before being fed into the function.


In [ ]:
# SET UP ENVIRONMENT 

# mount google drive for data storage and access
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#lib install
#!pip install PYCO2SYS

Mounted at /content/drive


In [ ]:
# IMPORTS

import os
import sys

import pandas as pd
import numpy as np

#import PyCO2SYS as pyco2


In [ ]:
# STORE DIRECTORY WHERE DATA IS STORED

# this is an absolute path 
# and be sure to include trailing '/'
data_dir = '/content/drive/MyDrive/Adam Gibbs/data/'
glider_dir = data_dir + 'glider/'

# Glider Data
The following cells parse the glider data files. This section is split up into the following three code cells:

1.   Reading Glider Data File Function
2.   Display Raw Data
3.   Parsing Function
4.   Parse Data and Create Input Array



In [ ]:
# FUNCTION TO READ GLIDER FILE IN A PANDAS DATAFRAME
def read_glider_file(file):
  names = [ 'Cruise', 'Station', 'Type',	'DATE', 'TIME', 'LONGITUDE',	
         'LATITUDE',	'QF',	'PRS', 'PRS_QF', 'TMP', 'TMP_QF', 
         'SAL', 'SAL_QF', 'Sigma_theta', 'ST_QF', 'DEPTH', 'DEPTH_QF', 
         'OXYGEN', 'OXYGEN_QF',	'SATOXY',	'SATOXY_QF',	'NITRATE', 
         'NITRATE_QF', 'CHL_A', 'CHL_A_QF', 'BBP700', 'BBP700_QF', 'PH_INSITU', 
         'PH_INSITU_QF', 'BBP532', 'BBP_532_QF', 'CDOM', 'CDOM_QF', 'TALK_CANYONB',	
         'TALK_QF', 'DIC_CANYONB', 'DIC_QF', 'pCO2_CANYONB', 'pCO2_QF', 
         'SAT_AR_CANYONB', 'SAT_AR_QF', 'pH25C_1atm', 'pH25C_1atm_QF' ]

  df = pd.read_csv(file, header=0, sep=',')
  df = df.dropna(axis=0, how='any').reset_index(drop=True)
  
  return df


In [ ]:
# READ DATA INTO A PANDAS DATAFRAME TO DISPLAY

print('Files in glider data folder:')
display(os.listdir(glider_dir))

dfs_glider = []
for file in os.listdir(glider_dir):
  if file == 'raw':
    continue
  dfs_glider.append(read_glider_file(glider_dir + file))

print()
print("There are {0} dataframes from {0} shipboard data files".format(len(dfs_glider)))
print("Adjust the index below to toggle which one is displayed.")
print()
display(dfs_glider[0])

Files in glider data folder:


['raw',
 '19502901.csv',
 '19402901.csv',
 '19502902.csv',
 '19A02901.csv',
 '20202901.csv',
 '19702901.csv']


There are 6 dataframes from 6 shipboard data files
Adjust the index below to toggle which one is displayed.



,Cruise,Station,Type,DATE,TIME,LONGITUDE,LATITUDE,QF,PRS,PRS_QF,TMP,TMP_QF,SAL,SAL_QF,Sigma_theta,ST_QF,DEPTH,DEPTH_QF,OXYGEN,OXYGEN_QF,SATOXY,SATOXY_QF,NITRATE,NITRATE_QF,CHL_A,CHL_A_QF,BBP700,BBP700_QF,PH_INSITU,PH_INSITU_QF,BBP532,BBP_532_QF,CDOM,CDOM_QF,TALK_CANYONB,TALK_QF,DIC_CANYONB,DIC_QF,pCO2_CANYONB,pCO2_QF,SAT_AR_CANYONB,SAT_AR_QF,pH25C_1atm,pH25C_1atm_QF
0,19502901,1,C,05/07/2019,17:18,-121.892,36.792,0,56.52,0.0,10.187,0.0,33.708,0.0,26.167,0.0,56.095,0.0,192.38,8.0,69.7,8.0,-1.000000e+10,1.0,0.448,0.0,-1.000000e+10,1.0,7.8538,8.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2251.0,8.0,2147.0,8.0,640.2,8.0,1.315,8.0,7.6428,8.0
1,19502901,1,C,05/07/2019,17:18,-121.892,36.792,0,56.28,0.0,10.186,0.0,33.708,0.0,26.166,0.0,55.857,0.0,184.58,8.0,66.9,8.0,-1.000000e+10,1.0,0.445,0.0,-1.000000e+10,1.0,7.8538,8.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2251.0,8.0,2146.0,8.0,639.9,8.0,1.315,8.0,7.6428,8.0
2,19502901,1,C,05/07/2019,17:18,-121.892,36.792,0,56.08,0.0,10.189,0.0,33.707,0.0,26.164,0.0,55.659,0.0,178.56,8.0,64.7,8.0,-1.000000e+10,1.0,0.440,0.0,-1.000000e+10,1.0,7.8531,8.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2250.0,8.0,2146.0,8.0,641.0,8.0,1.312,8.0,7.6421,8.0
3,19502901,1,C,05/07/2019,17:18,-121.892,36.792,0,55.68,0.0,10.190,0.0,33.706,0.0,26.161,0.0,55.262,0.0,174.18,8.0,63.1,8.0,-1.000000e+10,1.0,0.440,0.0,-1.000000e+10,1.0,7.8522,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2249.0,8.0,2146.0,8.0,642.3,8.0,1.309,8.0,7.6413,8.0
4,19502901,1,C,05/07/2019,17:18,-121.892,36.792,0,55.32,0.0,10.192,0.0,33.706,0.0,26.159,0.0,54.905,0.0,170.33,8.0,61.7,8.0,-1.000000e+10,1.0,0.471,0.0,-1.000000e+10,1.0,7.8515,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2249.0,8.0,2145.0,8.0,643.2,8.0,1.308,8.0,7.6407,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44505,19502901,83,C,05/17/2019,15:48,-122.031,36.746,0,3.44,0.0,14.560,0.0,33.199,0.0,24.693,0.0,3.415,0.0,263.50,0.0,104.2,0.0,-1.000000e+10,1.0,0.621,0.0,-1.000000e+10,1.0,8.0745,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2229.0,0.0,2015.0,0.0,363.8,0.0,2.368,0.0,7.9173,0.0
44506,19502901,83,C,05/17/2019,15:48,-122.031,36.746,0,2.44,0.0,14.568,0.0,33.198,0.0,24.686,0.0,2.422,0.0,263.39,0.0,104.1,0.0,-1.000000e+10,1.0,0.569,0.0,-1.000000e+10,1.0,8.0742,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2229.0,0.0,2015.0,0.0,364.1,0.0,2.367,0.0,7.9171,0.0
44507,19502901,83,C,05/17/2019,15:48,-122.031,36.746,0,1.56,0.0,14.565,0.0,33.198,0.0,24.683,0.0,1.548,0.0,263.43,0.0,104.1,0.0,-1.000000e+10,1.0,0.588,0.0,-1.000000e+10,1.0,8.0746,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2229.0,0.0,2015.0,0.0,363.7,0.0,2.369,0.0,7.9174,0.0
44508,19502901,83,C,05/17/2019,15:48,-122.031,36.746,0,0.60,0.0,14.559,0.0,33.199,0.0,24.681,0.0,0.596,0.0,263.32,0.0,104.1,0.0,-1.000000e+10,1.0,0.586,0.0,-1.000000e+10,1.0,8.0760,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2229.0,0.0,2014.0,0.0,362.4,0.0,2.375,0.0,7.9187,0.0


In [ ]:
# CREATE FUNCTION TO CREATE A NUMPY ARRAY OF INPUTS FROM
# GLIDER DATA FILE

'''
process_glider_input()
  description:
    This function reads in a data file in csv format and
    creates a pandas dataframe from it. From there it loops through
    and removes all bad data points according to the quality control
    flags. It then takes the desired input parameters as sepcified on 
    line 84 and puts them into a numpy array.

  args:
    file: string that contains file name of dataset
'''
def process_glider_file(file, save_txt=False, save_csv=False):

  # read in csv
  df = read_glider_file(file)

  # throw away first day
  start_date = int(df['DATE'][0][3:5])
  start_time = float(df['TIME'][0][0:2]) + float(df['TIME'][0][3:5]) / 60

  drop_index = 0
  for index, row in df.iterrows():
    curr_date = int(row['DATE'][3:5])
    curr_time = float(row['TIME'][0:2]) + float(row['TIME'][3:5]) / 60
    if (curr_date > start_date and curr_time > start_time) or curr_date > start_date + 1:
      drop_index = index
      break

  # drop first day of data
  df = df.drop(index=df.index[:drop_index], axis=0).reset_index(drop=True)

  # take only data we care about
  df = df[['DATE', 'LATITUDE', 'LONGITUDE', 'PRS', 'PRS_QF', 'TMP', 'TMP_QF',
          'SAL', 'SAL_QF', 'OXYGEN', 'OXYGEN_QF', 'SATOXY', 'SATOXY_QF',
          'PH_INSITU', 'PH_INSITU_QF', 'TALK_CANYONB', 'TALK_QF', 'DIC_CANYONB',
          'DIC_QF', 'pCO2_CANYONB', 'pCO2_QF']]

  # drop bad pressure
  index = 0
  to_drop = []
  for flag in df['PRS_QF']:
    if flag > 0:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad temperature
  index = 0
  to_drop = []
  for flag in df['TMP_QF']:
    if flag > 0:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad salinity
  index = 0
  to_drop = []
  for flag in df['SAL_QF']:
    if flag > 0:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad oxygen
  index = 0
  to_drop = []
  for flag in df['OXYGEN_QF']:
    if flag > 0:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad saturated oxygen
  index = 0
  to_drop = []
  for flag in df['SATOXY_QF']:
    if flag > 0:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad pH
  index = 0
  to_drop = []
  for flag in df['PH_INSITU_QF']:
    if flag > 0:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # take subset of only parameters for inputs
  # this array contains only "good" data points
  inputs = df[['DATE', 'LATITUDE', 'LONGITUDE', 'PRS', 'TMP', 'SAL', 'OXYGEN']]
  outputs = df['PH_INSITU']
  # convert dataframe in numpy array
  inputs = inputs.to_numpy(dtype='str')
  outputs = outputs.to_numpy(dtype='str')

  # change date format
  for row in inputs:
    date = row[0]
    row[0] = date[6:10] + date[0:2] + date[3:5]

  # return the array
  return inputs, outputs


'''
process_glider_dir()
  description:
    This function takes a dir with shipboard data and processes
    each file in that dir and creates input and output files. It
    also has the option to save those input and output arrays to
    either a .txt or a .csv file.

  args:
    file: string that contains file name of dataset
    save_txt (default=False): boolean of whether to save a .txt
    save_csv (default=False): boolean of whether to save a .csv
'''
def process_glider_dir(dir, save_txt=False, save_csv=False):
  # list all data files in directory
  print('Parsing the following files:')
  print(os.listdir(glider_dir))
  print()

  # create an empty numpy array that will hold inputs
  inputs = np.empty((0,7))
  outputs = np.empty(0)

  # loop through all data files and add them to input array
  for file in os.listdir(glider_dir):
    if file == 'raw':
      continue
    input_array, output_array = process_glider_file(glider_dir + file)
    inputs = np.concatenate((inputs, input_array), axis=0)
    outputs = np.concatenate((outputs, output_array), axis=0)

  # if desired save arrays as .txt files
  if save_txt or save_csv: 
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE PRS TMP SAL OXYGEN'
      np.savetxt(data_dir + 'glider_tpso_input.txt', inputs, 
                 fmt='%s', header=input_header)
      output_header = 'pH'
      np.savetxt(data_dir + 'glider_ph_output.txt', outputs, 
                 fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, PRS, TMP, SAL, OXYGEN'
      np.savetxt(data_dir + 'glider_tpso_input.csv', inputs, 
                 fmt='%s', delimiter=",", header=input_header)
      output_header = 'pH'
      np.savetxt(data_dir + 'glider_ph_output.csv', outputs, 
                 fmt='%s', delimiter=",", header=output_header)

  return inputs, outputs


In [ ]:
# CREATE INPUT FILE FROM GLIDER DATA

glider_inputs, glider_outputs = process_glider_dir(glider_dir, save_csv=True)

# print input array and its dimensions
print('Input array:')
display(glider_inputs)
print()

print('Dimensions of input array:')
display(glider_inputs.shape)
print()

# print output array and its dimensions
print('Output array:')
display(glider_outputs)
print()

print('Dimensions of output array:')
display(glider_outputs.shape)
print()

Parsing the following files:
['raw', '19502901.csv', '19402901.csv', '19502902.csv', '19A02901.csv', '20202901.csv', '19702901.csv']

Input array:


array([['20190508', '36.736', '-122.011', ..., '6.017',
        '34.233000000000004', '19.45'],
       ['20190508', '36.736', '-122.011', ..., '6.026', '34.232',
        '19.42'],
       ['20190508', '36.736', '-122.011', ..., '6.035', '34.231',
        '19.42'],
       ...,
       ['20190905', '36.796', '-121.838', ..., '13.873', '33.63',
        '247.89'],
       ['20190905', '36.796', '-121.838', ..., '14.398', '33.611',
        '266.55'],
       ['20190905', '36.796', '-121.838', ..., '14.405', '33.622',
        '281.73']], dtype='<U32')


Dimensions of input array:


(642065, 7)


Output array:


array(['7.5485', '7.5486', '7.5484', ..., '8.0313', '8.0642', '8.1087'],
      dtype='<U32')


Dimensions of output array:


(642065,)

In [ ]:
count = 0
for o in glider_outputs:
  if float(o) < 7.3 or float(o) > 8.5:
    count += 1
display(count)

0